# Path finding problem using uninformed and informed searchs

### Goal :
in this project, our goal is to compare some search algorithms to find the best solution according to their optimality, time complexity and memory complexity.

### Description :
in this project, we are going to compare 4 different search methods; first one is bredth fiers serch, second method is iterative deeping search, third method is A* search and the last one is weighted A* with different alpha weights. first we will describe and implement our states, initial state, goal state and actions. after describing these parameters and simiulating and formulazing our problem, then we implement these datas with class and nodes and try to describe our algorithms, depend on what order should we check all nodes and expand them.
at last we are going to calculate the time duration for each test and algorythms to see which algorithm is faster and better.

In [10]:
import numpy as np
from time import time
import copy
from heapq import heappush, heappop
import bisect

In [11]:
row = (1, 0, -1, 0)
col = (0, 1, 0, -1 )
doctors =tuple([(0,0)])
init = ((0,0),)

first we need to initialize some variables that we are going to use them further in our problem solvings. the two arrays rows and col are defined to implement the doctor's movements. for example if we add row[3] and col[3] to a doctor's location, it would be like moving the doctor one position to upper position. init array is where the first doctor is at the beginning of the queue. then we need an array for doctors to save their location as an ordered pair of the row and column indexes.

In [65]:
class Queue:  
    def __init__(self):
        self.heap = []
    
    def __iter__(self):
        for x in self.heap:
            yield x
    
    def push(self, element):
        bisect.insort(self.heap, element)

    def pop(self):
        return heappop(self.heap)
    
    def empty(self):
        if not self.heap:
            return True
        else:
            return False

we need to store all our nodes in somewhere so that we can do a search over them. for making the searching faster, we are using heap to store all our node and we name it as slass Queue. in this case we define some functions and method for adding a new node and removing a node from our heap. also we need to define iteration method over this class in order to make an explored list and check if a node is in this queue or not. at last for cheking if the queue is empty or not we define emty function.

In [64]:
class node:
    def __init__(self, parent, doctors, potions, duplicators, cost, level):  
        self.parent = parent
        self.doctors = doctors
        self.potions = potions
        self.duplicators = duplicators
        self.cost = cost
        self.level = level
    
    def __hash__(self):
        return hash((self.doctors,self.potions, self.duplicators))
        
    def __eq__(self, other): 
        if not isinstance(other, node):
            return NotImplemented
        return self.doctors == other.doctors and self.potions == other.potions and self.duplicators == other.duplicators
    
    def __lt__(self, nxt):
        return self.level + len(self.potions) + len(self.doctors) < nxt.level +  len(nxt.potions) + len(nxt.doctors)

### states and actions
in this part we are going to describe our states and actions. our goal in this problem is to collect all potions in the map and finally gather all doctors that exist in the map at location (n-1, m-1) wich n and m are the number of rows and columns respectively. in order to implement our states, we try to consider every embedding situation of all elements in the map, as a state and we assign a node to each state. we implement our map by defining a class call node wich contains some datas and variables. this class contains an array of doctors locations, array of potions locations, array of duplicators. in this case every single map would be correspondent to a single node. also we would store every states's cost to check if the state is the goal stat or not. also for BFS algorithm we need to know wich level we are, so we keep every node's level in this class too. the level variable means how many moves are needed to be take for doctors movements in case to reach this state. in this case, our initial state involves a doctors array wich only has a single member and that member is (0, 0) wich means the initial position for the doctor. the duplicators array and potions array are formed based on input, we receive these locations from input and we form these arrays. and our final state would be the node wich it's potions array is emty and also the doctor's array's memebers are all (n-1,m-1). and at last our actions are moving one of the existing doctors in the map to either up, down, left or right. the __it__ and __eq__ methods are defined so that we can store them bycomparing something in our heap. generally we compare two nodes based on how much memory the take. so we add all the arrays length together and compare the result. also we consider to nodes are equal if theire doctors, potions and duplicators are the same.

In [14]:
def calculateCost(doctors, potions, final) -> int: 
    count = 0
    for doc in doctors:
        if doc != final:
            count += 1
    count += len(potions)
    return count

calculatecost function would calculate the cost for each state. it matters only when it become 0 wich means we have reached to goal state. so we need to calculate it in the way such that the cost is zero for the gola(final) states and nonzero for all other states. in this case we calculate the important variables in achieving to goal state. wich are number of potions that it should be zero at the goal state and also doctors locations that each of them shold be equal to final (n-1,m-1).

In [15]:
def isSafe(x, y, walls,final):
    if (x,y) in walls:
        return False
    return x >= 0 and x <= final[0] and y >= 0 and y <= final[1]

is safe dunction checks if an action is valid or not. in other words it would check if the movement is in the map frame or not and also if ther isn't a wall in the frontier tile.

In [16]:
def printPath(root):
    if root.parent == None:
        return
    printPath(root.parent)
    print(root.level,"- doctors locations : ", root.doctors)

prithpath function use a recursive method to print the path all the way from initial state to goal state.

In [17]:
def newNode(doctors, doc_loc, duplicators, potions, parent, level, final) -> node:
    [x, y, i] = doc_loc
    n = final[0]
    new_doc = list(copy.deepcopy(doctors))
    new_pot = set(copy.deepcopy(potions))
    new_dup = set(copy.deepcopy(duplicators))
    new_par = copy.deepcopy(parent)
    new_doc[i] = (x, y)
    if new_doc[i] in new_dup:
        new_dup.remove(new_doc[i]) 
        new_doc.append((n, 0))
    if (n, 0) in new_pot:
        new_pot.remove((n,0))
    if (0,0) in new_dup:
        new_dup.remove((0,0))
        new_doc.append((n,0))
    if new_doc[i] in new_pot:
        new_pot.remove(new_doc[i])
    cost = calculateCost(new_doc, new_pot, final)
    new_node = node(new_par, tuple(new_doc), tuple(new_pot), tuple(new_dup), cost, level)
    return new_node

newNode function creates  a new live node and return the node so that we can use and check this state in the future. at first we get an 1* 3 array called doc_loc as an input. this array contains the x and y of the new location in the map that we want to move a doctor to and also the index of the doctor wich means that which dooctor we want to be moved to location (x,y). then we have to check if by moving a doctor to this tile, anything would happen to other variable arrays in a node class or not. in the other word we should check if there exists any potions or duplicators in the destination tile. after applying all these changes to new arrays we would calculate the cost and make a new map and add it to our queue.

In [62]:
def solveBFS(doctors, potions,walls,duplicators, final):
    pq = Queue()
    explored = set()
    cost = calculateCost(doctors, potions, final)
    root = node(None, doctors, potions, duplicators, cost, 0)
    pq.push(root)
    while not pq.empty():
        minimum = pq.pop()
        pre_size = len(explored)
        explored.add(minimum)
        nxt_size = len(explored)
        if pre_size != nxt_size :
            if minimum.cost == 0: 
                return minimum
            for i in range(len(minimum.doctors)):
                for j in range(3):
                    doc_loc = [minimum.doctors[i][0] +  row[j], minimum.doctors[i][1] + col[j], i]
                    if isSafe(doc_loc[0], doc_loc[1], walls,final):
                        child = newNode(tuple(minimum.doctors),doc_loc, tuple(minimum.duplicators), tuple(minimum.potions), minimum, minimum.level + 1, final)   
                        if child.cost == 0:
                            return child
                        if child not in explored:
                            if child not in pq:
                                pq.push(child)

now we are about to implement the BFS algorithm search. in this case first we define a new emty queue. also we define a set for explored node to make our search faster. then we have to make the root of this queue wich is the initial state. after that we try to find the goal state in a loop and every wingle time we get in to the end of the loop, we go to the nex level of nodes and we try to search for the goal state in there. for finding and making new nodes we get the very first nod in the frontier (wich is the most left node with the lowest level that hasn't expanded yet.) we create all it's cgildrens. for doing so se need to move all the existing doctors in the map in to all possible directions. then every time we make chils we check if it is in the explored or not and then we would check the cost. we will continue doing this until we find a node that has e 0 cost. (for checking if a node is explored before or not we consider a set o nodes and every time we make a new child and check its cost, we would add that node to this function and if the size of this set didn't change, it means ths node was already in the queue and we would skip it.

In [51]:
def calculate_BFS():
    n , m= list(map(int, input().split()))
    c , k = list(map(int, input().split()))

    potions = []
    for i in range(c):
        potions.append(tuple(map(int, input().split())))

    potions = tuple(potions)

    duplicators = []
    for i in range(k):
        duplicators.append(tuple(map(int, input().split())))
    duplicators = tuple(duplicators)

    walls = []
    d =int(input())
    for i in range(d): 
        walls.append(tuple(map(int, input().split())))
    walls = tuple(walls)
    final = (n-1,m-1)

    t1 = time()
    minimum = solveBFS(doctors, potions,walls,duplicators, final)
    t2 = time()
    print("0 - doctors locations : ", init)
    printPath(minimum)
    print(t2-t1)

at last we need to define a function in order to get inputs and calculate and find the path to goal state and also record the time duration of each test and compare these numbers at the end of the project.

In [65]:
calculate_BFS()

4 4
2 1
0 2
2 0
0 1
1
1 2
0 - doctors locations :  ((0, 0),)
1 - doctors locations :  ((0, 1), (3, 0))
2 - doctors locations :  ((0, 1), (2, 0))
3 - doctors locations :  ((0, 2), (2, 0))
4 - doctors locations :  ((0, 2), (3, 0))
5 - doctors locations :  ((0, 3), (3, 0))
6 - doctors locations :  ((1, 3), (3, 0))
7 - doctors locations :  ((1, 3), (3, 1))
8 - doctors locations :  ((2, 3), (3, 1))
9 - doctors locations :  ((2, 3), (3, 2))
10 - doctors locations :  ((2, 3), (3, 3))
11 - doctors locations :  ((3, 3), (3, 3))
0.27829480171203613


In [66]:
calculate_BFS()

4 4
3 3
1 2
2 0
2 1
0 0
1 1
3 1
2
0 3
1 3
0 - doctors locations :  ((0, 0),)
1 - doctors locations :  ((0, 1), (3, 0))
2 - doctors locations :  ((0, 1), (2, 0))
3 - doctors locations :  ((0, 2), (2, 0))
4 - doctors locations :  ((1, 2), (2, 0))
5 - doctors locations :  ((1, 2), (2, 1))
6 - doctors locations :  ((1, 2), (2, 2))
7 - doctors locations :  ((1, 2), (2, 3))
8 - doctors locations :  ((2, 2), (2, 3))
9 - doctors locations :  ((2, 2), (3, 3))
10 - doctors locations :  ((2, 3), (3, 3))
11 - doctors locations :  ((3, 3), (3, 3))
10.728812456130981


In [75]:
calculate_BFS()

6 6
5 2
1 2
1 5
3 5
4 3
5 1
2 2
4 2
4
0 2
1 1
4 1
3 0
0 - doctors locations :  ((0, 0),)
1 - doctors locations :  ((1, 0),)
2 - doctors locations :  ((2, 0),)
3 - doctors locations :  ((2, 1),)
4 - doctors locations :  ((2, 2), (5, 0))
5 - doctors locations :  ((2, 2), (5, 1))
6 - doctors locations :  ((1, 2), (5, 1))
7 - doctors locations :  ((1, 3), (5, 1))
8 - doctors locations :  ((1, 4), (5, 1))
9 - doctors locations :  ((1, 5), (5, 1))
10 - doctors locations :  ((1, 5), (5, 2))
11 - doctors locations :  ((1, 5), (5, 3))
12 - doctors locations :  ((2, 5), (5, 3))
13 - doctors locations :  ((2, 5), (4, 3))
14 - doctors locations :  ((2, 5), (4, 4))
15 - doctors locations :  ((3, 5), (4, 4))
16 - doctors locations :  ((3, 5), (5, 4))
17 - doctors locations :  ((4, 5), (5, 4))
18 - doctors locations :  ((4, 5), (5, 5))
19 - doctors locations :  ((5, 5), (5, 5))
131.30475187301636


as you can see we tested all 3 test using this BFS algorythm and as it is shown the duration of these test has increased way to much, every time the map gets more complicated and as you can see the last one took 130 seconds to find the shortest path to the goal state.

In [104]:
 def DLS(node,maxDepth, walls, final, explored):
        if maxDepth <= 0 : return None
        for i in range(len(node.doctors)):
            for j in range(3):
                doc_loc = [node.doctors[i][0] +  row[j], node.doctors[i][1] + col[j], i]
                if isSafe(doc_loc[0], doc_loc[1], walls,final):
                    child = newNode(tuple(node.doctors),doc_loc, tuple(node.duplicators), tuple(node.potions), node, node.level + 1, final)
                    if child.cost == 0:
                        return child
                    if child not in explored:
                        explored.push(child)
                        result = DLS(child, maxDepth-1, walls, final, explored)
                        if result != None:
                            return result

then we implemented the IDL algorithm. this time we didnt define any queue for keeping all the nodes in it. becase in this algorithm we use an recursive method and we would be able to access to all fathers node(we wont need to store childs becabecause all of the node's child will be expanded sooner than their father, but we still have the explored set in order to reduce the search time duration. in this method we would use DLS function but with a limited debth and hen we would increase the depth one by one until we achive to the goal state. every time we want to go to the deepest node, we call the DFS function for the child of that node but under one level lower maximum depth limit. and in this case we are implementing the DFS limited algorythm.

In [108]:
def solveIDL(doctors, potions,walls,duplicators, final):
    cost = calculateCost(doctors, potions, final)
    root = node(None, doctors, potions, duplicators, cost, 0)
    maxDepth = 0
    result = None
    while result == None:
        explored = Queue()
        result = DLS(root, maxDepth, walls, final, explored)
        maxDepth +=1
    return result

the by using a loop over maxdepth and calling the DLS function, we have just implemented the IDL search. 

In [106]:
def calculate_IDL():
    n , m= list(map(int, input().split()))
    c , k = list(map(int, input().split()))

    potions = []
    for i in range(c):
        potions.append(tuple(map(int, input().split())))

    potions = tuple(potions)

    duplicators = []
    for i in range(k):
        duplicators.append(tuple(map(int, input().split())))
    duplicators = tuple(duplicators)

    walls = []
    d =int(input())
    for i in range(d): 
        walls.append(tuple(map(int, input().split())))
    walls = tuple(walls)
    final = (n-1,m-1)

    t1 = time()
    minimum = solveIDL(doctors, potions,walls,duplicators, final)
    t2 = time()
    print("0 - doctors locations : ", init)
    printPath(minimum)
    print(t2-t1)

at the end we call this function over the test number one, as you can see it tooks more time to find the path and achive to goal state. but at the end it has founded an excepted result.

In [113]:
calculate_IDL()

4 4
2 1
0 2
2 0
0 1
1
1 2
0 - doctors locations :  ((0, 0),)
1 - doctors locations :  ((1, 0),)
2 - doctors locations :  ((2, 0),)
3 - doctors locations :  ((3, 0),)
4 - doctors locations :  ((3, 1),)
5 - doctors locations :  ((2, 1),)
6 - doctors locations :  ((1, 1),)
7 - doctors locations :  ((0, 1), (3, 0))
8 - doctors locations :  ((0, 2), (3, 0))
9 - doctors locations :  ((0, 3), (3, 0))
10 - doctors locations :  ((1, 3), (3, 0))
11 - doctors locations :  ((2, 3), (3, 0))
12 - doctors locations :  ((3, 3), (3, 0))
13 - doctors locations :  ((3, 3), (3, 1))
14 - doctors locations :  ((3, 3), (3, 2))
15 - doctors locations :  ((3, 3), (3, 3))
0.8886709213256836


### result

if we compare all the time durations of these algorithms we can see informed seachs are mostly so much faster thar uninformed searches. and also in informed search, weighted A* search is faster that normal A* seach. and between uninformed searches, BFS seems to be a little bit faster that the IDL because in IDL, every node will be explore every time the limit for depth increases. any of other algorythms may be faster that others in a few test but generally informed searches are way too faster and also take less memory.


#### References 

1-https://www.geeksforgeeks.org/8-puzzle-problem-using-branch-and-bound/

2-https://www.tutorialspoint.com/python_data_structure/python_queue.htm

3-https://www.geeksforgeeks.org/taking-multiple-inputs-from-user-in-python/

4-https://stackoverflow.com/questions/6310867/why-arent-python-sets-hashable

5-https://stackoverflow.com/questions/1227121/compare-object-instances-for-equality-by-their-attributes

6-https://www.geeksforgeeks.org/iterative-deepening-searchids-iterative-deepening-depth-first-searchiddfs/